# Test the VGG Net
reference: https://076923.github.io/posts/Python-pytorch-10/

## Call the Model

In [26]:
from collections import OrderedDict

import torch
from torch import nn
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

In [2]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.layer = nn.Linear(2, 1)
    
    def forward(self, x):
        x = self.layer(x)
        return x

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)

model_state_dict = torch.load("./model/vgg_net.pt", map_location=device)
model.load_state_dict(model_state_dict)

RuntimeError: Error(s) in loading state_dict for CustomModel:
	Missing key(s) in state_dict: "layer.weight", "layer.bias". 
	Unexpected key(s) in state_dict: "model", "epoch", "model_state_dict", "optimizer_state_dict", "cost", "description". 

In [19]:
model.load_state_dict(torch.load("./model/vgg_net.pt", map_location=device))

RuntimeError: Error(s) in loading state_dict for CustomModel:
	Missing key(s) in state_dict: "layer.weight", "layer.bias". 
	Unexpected key(s) in state_dict: "model", "epoch", "model_state_dict", "optimizer_state_dict", "cost", "description". 

In [32]:
model_state_dict.keys()

dict_keys(['model', 'epoch', 'model_state_dict', 'optimizer_state_dict', 'cost', 'description'])

In [27]:
model_state_dict = torch.load("./model/vgg_net.pt", map_location=device)
new_state_dict = OrderedDict()

for n, v in model_state_dict.items():
    name = n.replace("module.", "")
    new_state_dict[n] = v

In [29]:
model.load_state_dict(new_state_dict)

RuntimeError: Error(s) in loading state_dict for CustomModel:
	Missing key(s) in state_dict: "layer.weight", "layer.bias". 
	Unexpected key(s) in state_dict: "model", "epoch", "model_state_dict", "optimizer_state_dict", "cost", "description". 

In [22]:
model_state_dict["epoch"]

7

## Load Test Dataset

In [9]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                               ])

testset = torchvision.datasets.ImageFolder(root="../data/fer2013/test",
                                           transform=transform)

In [11]:
print(testset)

Dataset ImageFolder
    Number of datapoints: 7178
    Root location: ../data/fer2013/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [12]:
classes = testset.classes
print(classes)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [13]:
batch_size = 16

test_loader = DataLoader(testset, 
                         batch_size=batch_size,
                         shuffle=True)

In [15]:
correct = 0 # 맞은 개수
total = 0   # 전체 개수

# no_grad
with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y = label.to(device)
        
        # 모델에 데이터를 넣고 결과값 저장
        output = model(forward(x))
        _, output_index = torch.max(output, 1)
        
        # 전체 개수 += label 개수
        total += label.size(0)
        correct += (output_index == y).sum().float()
        
    # accuracy 
    print("Accuracy of Test Data: {}%".format(100 * correct / total))

NameError: name 'forward' is not defined